**Aviso Legal: Este notebook tem um fim única e exclusivamente didático, para alunos da FATEC, como uma introdução à Análise de Dados Públicos. Desse modo, está excluída a garantia ou responsabilidade de qualquer tipo, por exemplo, de precisão, confiabilidade, completude e atualidade das informações.**

Fonte dos dados : https://opendatasus.saude.gov.br/dataset/bd-srag-2020



In [45]:
import pandas as pd
import xlrd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import os
import shutil
from pathlib import Path
import pathlib
import time

In [10]:
### Iniciando o modo offline do Plotly
py.init_notebook_mode(connected=True)

In [11]:
if not os.path.exists("images"):
    os.mkdir("images")

In [12]:
cols = ['SEM_NOT', 'DT_NOTIFIC', 'SG_UF_NOT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'CS_GESTANT', 'CS_RACA', 'CS_ETINIA', 'CS_ESCOL_N', 'SG_UF', 'CLASSI_FIN', 'NOSOCOMIAL', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_SIN', 'OUTRO_DES', 'CLASSI_OUT', 'EVOLUCAO', 'PCR_RESUL', 'DS_IF_OUT','IF_RESUL','PCR_SARS2','DS_PCR_OUT','PCR_OUTRO']

In [13]:
df = pd.read_csv(r'arquivos\INFLUD-23-06-2020.csv', usecols=cols, delimiter=';', encoding='ISO-8859-1')

d:\mylib\pessoal\estudos\fatec\bd\6semestre\bd_nao_estruturado\analise-dados-publicos-covid19\venv\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning:

Columns (18,119) have mixed types.Specify dtype option on import or set low_memory=False.



In [14]:
# Tabelas de microdados usam números, faço a correspondência desses números com seus significados
troca_pcr_res = {1:'Detectável', 2:'Não Detectável', 3:'Inconclusivo',4:'Aguardando Resultado', 5:'Aguardando Resultado', 9:'Ignorado'}
troca_if_res = {1:'Positivo', 2:'Negativo', 3:'Inconclusivo',4:'Não Realizado', 5:'Aguardando', 9:'Ignorado'}
troca_raca = {1:'Branca', 2:'Preta', 3:'Amarela', 4:'Parda', 5:'Indígena', 9:'Ignorado'}
troca_escolaridade = {0:'Sem', 1:'Fund1', 2:'Fund2', 3:'Médio', 4:'Superior', 5:'Não se aplica', 9:'Ignorado'}

In [15]:
#Trocando valores vazios para ignorados
df.loc[(df.CS_RACA.isnull()), 'CS_RACA'] = 9
df.loc[(df.CS_ESCOL_N.isnull()), 'CS_ESCOL_N'] = 9
df.loc[(df.PCR_RESUL.isnull()), 'PCR_RESUL'] = 9
df.loc[(df.IF_RESUL.isnull()), 'IF_RESUL'] = 9

#Aplicando os valores do dicionario de dados
df['CS_RACA'] = df['CS_RACA'].apply(lambda x: troca_raca[x])
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].apply(lambda x: troca_escolaridade[x])
df['PCR_RESUL'] = df['PCR_RESUL'].apply(lambda x: troca_pcr_res[x])
df['IF_RESUL'] = df['IF_RESUL'].apply(lambda x: troca_if_res[x])

df.SEM_NOT = df.SEM_NOT.apply(pd.to_numeric, errors='coerce')
df['DT_NOTIFIC'].dropna(inplace=True)

In [ ]:
#filtra os valores relacionados a COVID-19
df = df.query("CLASSI_FIN == 5 or CLASSI_OUT.str.contains('CORONA|COVID|COV|SARS|19') or DS_IF_OUT.str.contains('COV|SARS|CORONA') or DS_PCR_OUT.str.contains('COVID|SARS|COV2|19')", engine='python')

In [68]:
### Visualização de contágios por região
semana  = df.groupby('SEM_NOT').size().to_dict()

data = go.Bar(x = [*semana.keys()],
              y = [*semana.values()] )

layout = go.Layout(title='Total de contágios por semana',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Semana'})

fig = go.Figure(data=data, layout=layout)

### Gerando imagem do grafico
### O Plotly vai abrir o HTML numa nova aba e baixar o arquivo .png
py.plot(fig, image='png',image_filename='graf-semana', output_type='file', image_width=1000, image_height=500, filename='images/graf-semana.html', validate=False)

### Esperando gerar o arquivo
#time.sleep(3)

### Movemos o arquivo para nossa pasta do repositorio a fim de inserir a exibição de forma estática
#downloads_path = str(Path.home() / "Downloads/graf-semana.png")
#destination = os.path.abspath('') + r'\images' 
#shutil.move(downloads_path, destination)

'images/graf-semana.html'

![Semana](./images/graf-semana.png)

Obs.: A última semana deve ser ignorada, pois o fechamento do arquivo se dá antes da conclusão da semana e por isso os dados estão incompletos

In [ ]:
### Visualização de contágios por região
regiao = df.groupby('SG_UF_NOT').size().to_dict()

data = go.Bar(x = [*regiao.keys()],
              y = [*regiao.values()] )

layout = go.Layout(title='Total de contágios por região',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'UF'})

fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show()

In [ ]:
### Visualização de contágios por raça
raca = df.groupby('CS_RACA').size().to_dict()

data =  go.Bar(x = [*raca.keys()],
              y = [*raca.values()] )


layout = go.Layout(title='Total de contágios por raça',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Raça'})
                  
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show()

In [ ]:
### Visualização de contágios por nível de escolaridade
escolaridade = df.groupby('CS_ESCOL_N').size().to_dict()

data =  go.Bar(x = [*escolaridade.keys()],
              y = [*escolaridade.values()] )


layout = go.Layout(title='Total de contágio por nível de escolaridade',
                   yaxis={'title':'Quantidade'},
                   xaxis={'title': 'Nível de escolaridade'})
                  
fig = go.Figure(data=data, layout=layout)

# Exibindo figura/gráfico
fig.show()